In [1]:
pwd

'/'

In [2]:
import os
project_path = '/workspace/'
os.chdir(project_path)

In [3]:
!pip install hydra-core --upgrade
!pip install hdf5plugin

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [11]:
!sudo apt-get update && sudo apt-get install -y libgl1 libglib2.0-0


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1 is already the newest version (1.4.0-1).
The following additional packages will be installed:
  libglib2.0-data shared-mime-info xdg-user-dirs
The following NEW packages will be installed:
  libglib2.0-0 libglib2.0-data shared-mime-info xdg-user-dirs
0 upgraded, 4 newly installed, 0 to remove and 16 not upgraded.
Need to get 1978 kB of archives.
After this operation, 7664 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgl

In [12]:
%env HYDRA_FULL_ERROR=1
!python main.py

env: HYDRA_FULL_ERROR=1


Imageio: 'libfreeimage-3.16.0-linux64.so' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/freeimage/libfreeimage-3.16.0-linux64.so (4.6 MB)
Downloading: 8192/4830080 bytes (0.2%4830080/4830080 bytes (100.0%)
  Done
File saved as /home/ubuntu/.imageio/freeimage/libfreeimage-3.16.0-linux64.so.
on epoch: 1
  0%|                                                   | 0/252 [00:00<?, ?it/s]/home/ubuntu/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
batch 0 loss: 195.5083610647692
100%|█████████████████████████████████████████| 252/252 [31:51<00:00,  7.58s/it]
Epoch 1, Loss: 14.62558201219742
on epoch: 2
100%|██████